
# Random Forest Classification Model(Multiple) for MIT data 
As we concluded before, for MIT data, we apply the following preprocessing:   
Resampling: oversampling  
Scaling: minmax

If you don't have the original files: run the notebook `preprocessing_mit_multipleclass_minmax_oversampling.ipynb`   
Input file:(The preprocessed data)   
mitbih_train_multiple_minmax_oversampling.csv   
mitbih_test_multiple_minmax_oversampling.csv   

Output: Random Forest Tree model trained  
model_mit_random_forest_multiple.pkl  

In [1]:
import sys
import os 

data_path = ''
model_output_path = ''
# check if the enviorment is Google Colab 

if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    # Install required libraries
    !pip install scikit-learn -q
    !pip install pandas -q
    !pip install numpy -q
    !pip install imbalanced-learn -q

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    # set the path where the csv file stored in your google drive. 
    data_path = '/content/drive/MyDrive/Heartbeat_Project/'
    model_output_path = data_path

else:
    print("Running on local environment")

    current_path = os.getcwd()
    print("Current working directory:", current_path)
    data_path = '../data/processed/'
    model_output_path = '../models/'

   

Running on local environment
Current working directory: /Users/pingyuan/Documents/codeself/heartbeat-analysis-rd/notebooks


# Read Data

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_curve, auc
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

RawFiles = dict({
    'test': data_path + 'mitbih_test_multiple_minmax_oversampling.csv',
    'train': data_path + 'mitbih_train_multiple_minmax_oversampling.csv',
})

OutputFiles = dict({
    'model': model_output_path +  'model_mit_random_forest_multiple.pkl'
})


train = pd.read_csv(RawFiles.get('train'),sep=',',header=0)

test = pd.read_csv(RawFiles.get('test'),sep=',',header=0)

y_train = train['target']
X_train = train.drop('target', axis=1)

y_test = test['target']
X_test = test.drop('target', axis=1)



## Optimized Balanced Random Forest Classifier Grid Search

In [4]:
# Balanced Random Forest Classifier
rf_model = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

# Grid parameters for RandomForest
rf_grid_params = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for splitting a node
}


# Grid search for Balanced Random Forest
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_grid_params, cv=5, scoring='f1', n_jobs=-1)

# Fit the Random Forest grid search
rf_grid_search.fit(X_train, y_train)

# Get the best parameters and model for RF
rf_best_params = rf_grid_search.best_params_
print("Balanced RF Best parameters:", rf_best_params)

rf_best_score = rf_grid_search.best_score_
print("Balanced RF Best score:", rf_best_score)

rf_best_model = rf_grid_search.best_estimator_

# Predict and evaluate RF
y_pred_rf = rf_best_model.predict(X_test)
f1_rf = f1_score(y_test, y_pred_rf)
print("Balanced RF F1 score:", f1_rf)

conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Balanced RF Confusion Matrix')
plt.show()

class_report_rf = classification_report(y_test, y_pred_rf)
print("Balanced RF Classification Report:\n", class_report_rf)

if len(np.unique(y_test)) == 2:
    y_prob_rf = rf_best_model.predict_proba(X_test)[:, 1]
    fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
    roc_auc_rf = auc(fpr_rf, tpr_rf)

    plt.figure(figsize=(10, 6))
    plt.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label='Balanced RF ROC curve (area = %0.2f)' % roc_auc_rf)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Balanced RF Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.show()
else:
    print("ROC curve is only applicable for binary classification in Balanced RF.")


/Users/pingyuan/Documents/codeself/heartbeat-analysis-ai/.conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/pingyuan/Documents/codeself/heartbeat-analysis-ai/.conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pingyuan/Documents/codeself/heartbeat-analysis-ai/.conda/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pingyuan/Documents/codeself/heartbeat-analysis-ai/.conda/lib/python3.11/site-package

KeyboardInterrupt: 

# Save Best Random Forest (balanced) Model

In [6]:
# Save model 
import pickle
# Save the model to a file
with open(OutputFiles.get('model'), 'wb') as model_file:
    pickle.dump(rf_best_model, model_file)